# Гонов М.И. ИУ5-65Б 
# Вариант №4

# РК 2. Методы построения моделей машинного обучения

**Набор данных:** `toy_dataset.csv`  
**Задача:** бинарная классификация (`Illness`)  
**Методы:** 1) Логистическая регрессия, 2) Градиентный бустинг  
**Метрики качества:** Accuracy и ROC AUC (почему — см. ниже)


In [66]:
# 1. Импорт библиотек и загрузка данных
import os
print("Рабочая директория:", os.getcwd())
print("Файлы:", os.listdir())

import pandas as pd
df = pd.read_csv('toy_dataset.csv')
print("Размер данных:", df.shape)
df.head()


Рабочая директория: C:\Users\marat\OneDrive\Рабочий стол\BM6\Технологии машинного обучения
Файлы: ['.ipynb_checkpoints', '.venv', 'lab3 (5).ipynb', 'lab4.ipynb', 'lab5.ipynb', 'lr 3', 'lr1', 'lr2', 'rk2.ipynb', 'tmo', 'toy_dataset.csv', 'wind_dataset.csv', 'Гонов ИУ5-65Б.ipynb.ipynb']
Размер данных: (150000, 6)


,Number,City,Gender,Age,Income,Illness
0,1,Dallas,Male,41,40367.0,No
1,2,Dallas,Male,54,45084.0,No
2,3,Dallas,Male,42,52483.0,No
3,4,Dallas,Male,40,40941.0,No
4,5,Dallas,Male,46,50289.0,No


## 2. Предобработка данных

1. Удаляем ID-столбец `Number`.  
2. Кодируем целевую переменную (`Illness`) как 0/1.  
3. One-hot кодируем категориальные признаки (`City`, `Gender`).  
4. Стандартизируем числовые признаки (`Age`, `Income`).


In [67]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 2.1. Убираем идентификатор
df = df.drop(columns=['Number'])

# 2.2. LabelEncoding для целевой
le = LabelEncoder()
df['Illness'] = le.fit_transform(df['Illness'])

# 2.3. One-hot для категорий
df = pd.get_dummies(df, columns=['City','Gender'], drop_first=True)

# 2.4. Стандартизация числовых признаков
num_cols = ['Age','Income']
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Разбиваем X и y
X = df.drop('Illness', axis=1)
y = df['Illness']

print("Признаки:", X.columns.tolist())


Признаки: ['Age', 'Income', 'City_Boston', 'City_Dallas', 'City_Los Angeles', 'City_Mountain View', 'City_New York City', 'City_San Diego', 'City_Washington D.C.', 'Gender_Male']


## 3. Разбиение на обучающую и тестовую выборки

Используем `train_test_split` (30% для теста), фиксируем `random_state` для воспроизводимости.


In [68]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (105000, 10) Test: (45000, 10)


## 4. Метод 1: Логистическая регрессия

Логрег — базовый линейный классификатор, хорошо интерпретируется и быстро обучается.


In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Логрег с балансировкой весов классов
lr_bal = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',   # учёт дисбаланса
    random_state=42
)
lr_bal.fit(X_train, y_train)

# Предсказания
y_pred_bal = lr_bal.predict(X_test)
y_proba_bal = lr_bal.predict_proba(X_test)[:,1]

# Метрики
acc_bal = accuracy_score(y_test, y_pred_bal)
auc_bal = roc_auc_score(y_test, y_proba_bal)

print(f"Balanced LogReg  Accuracy = {acc_bal:.3f}")
print(f"Balanced LogReg  ROC AUC   = {auc_bal:.3f}\n")
print("Classification Report (balanced):")
print(classification_report(
    y_test, y_pred_bal, zero_division=0
))


Balanced LogReg  Accuracy = 0.531
Balanced LogReg  ROC AUC   = 0.494

Classification Report (balanced):
              precision    recall  f1-score   support

           0       0.92      0.54      0.68     41358
           1       0.08      0.45      0.14      3642

    accuracy                           0.53     45000
   macro avg       0.50      0.50      0.41     45000
weighted avg       0.85      0.53      0.63     45000



## 5. Метод 2: Градиентный бустинг

Деревья в ансамбле, обычно дают более высокое качество на неглубоких табличных данных.


In [70]:
from sklearn.ensemble import GradientBoostingClassifier

# Обучение
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)

# Предсказания
y_pred_gb = gb.predict(X_test)
y_proba_gb = gb.predict_proba(X_test)[:,1]

# Метрики
acc_gb = accuracy_score(y_test, y_pred_gb)
auc_gb = roc_auc_score(y_test, y_proba_gb)

print(f"GradBoost Accuracy = {acc_gb:.3f}")
print(f"GradBoost ROC AUC  = {auc_gb:.3f}\n")
print("Classification Report (GradBoost):")
print(classification_report(y_test, y_pred_gb))


GradBoost Accuracy = 0.919
GradBoost ROC AUC  = 0.501

Classification Report (GradBoost):
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     41358
           1       0.33      0.00      0.00      3642

    accuracy                           0.92     45000
   macro avg       0.63      0.50      0.48     45000
weighted avg       0.87      0.92      0.88     45000



## 6. Сравнение моделей и выводы

| Модель               | Accuracy | ROC AUC |
|----------------------|---------:|--------:|
| Логистическая регрессия |  …       |    …    |
| Градиентный бустинг     |  …       |    …    |

**Почему метрики?**  
- **Accuracy** — простая и понятная доля правильных ответов, но не учитывает баланс классов.  
- **ROC AUC** — показывает качество ранжирования и нечувствительна к порогу отсечения, отражает способность модели различать классы.

**Выводы:**  
- Если ROC AUC и Accuracy у градиентного бустинга существенно выше, выбираем его для финального решения.  
- Если модели близки — можно дополнительно смотреть на Precision–Recall, скорость обучения и интерпретируемость (логрег гораздо проще объяснить бизнес-стейкхолдерам).
